In [4]:
from alpha_vantage.timeseries import TimeSeries

def fetch_stock_data_alpha_vantage(ticker, api_key="1H4BLOX4T4ZVGAED"):
    ts = TimeSeries(key=api_key, output_format='pandas')
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize='compact')
        data.reset_index(inplace=True)
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

stock_data = fetch_stock_data_alpha_vantage("AAPL", api_key="1H4BLOX4T4ZVGAED")
print(stock_data.head())

        date  1. open  2. high  3. low  4. close   5. volume
0 2024-11-20   228.06   229.93  225.89    229.00  35169566.0
1 2024-11-19   226.98   230.16  226.66    228.28  36211774.0
2 2024-11-18   225.25   229.74  225.17    228.02  44686020.0
3 2024-11-15   226.40   226.92  224.27    225.00  47923696.0
4 2024-11-14   225.02   228.87  225.00    228.22  44923941.0


In [5]:
def calculate_rsi(data, period=14):
    """
    Calculate the Relative Strength Index (RSI) for a given dataset.
    :param data: DataFrame with a 'close' column
    :param period: Look-back period for RSI (default: 14)
    :return: DataFrame with an additional 'RSI' column
    """
    # Calculate daily price changes
    data["change"] = data["4. close"].diff()
    
    # Separate gains and losses
    data["gain"] = data["change"].apply(lambda x: x if x > 0 else 0)
    data["loss"] = data["change"].apply(lambda x: -x if x < 0 else 0)
    
    # Calculate average gain and loss
    data["avg_gain"] = data["gain"].rolling(window=period, min_periods=1).mean()
    data["avg_loss"] = data["loss"].rolling(window=period, min_periods=1).mean()
    
    # Calculate Relative Strength (RS)
    data["rs"] = data["avg_gain"] / data["avg_loss"]
    
    # Calculate RSI
    data["RSI"] = 100 - (100 / (1 + data["rs"]))
    
    return data

# Calculate RSI for the stock data
stock_data_with_rsi = calculate_rsi(stock_data)
stock_data_with_rsi.tail()

,date,1. open,2. high,3. low,4. close,5. volume,change,gain,loss,avg_gain,avg_loss,rs,RSI
95,2024-07-09,227.93,229.40,226.3721,228.68,48169822.0,-4.30,0.0,4.30,1.708571,0.962857,1.774481,63.957219
96,2024-07-08,227.09,227.85,223.2500,227.82,59085861.0,-0.86,0.0,0.86,1.708571,1.004286,1.701280,62.980516
97,2024-07-05,221.65,226.45,221.6500,226.34,60412408.0,-1.48,0.0,1.48,1.708571,1.076429,1.587259,61.349064
98,2024-07-03,220.00,221.55,219.0300,221.55,37369801.0,-4.79,0.0,4.79,1.633571,1.418571,1.151561,53.522116
99,2024-07-02,216.15,220.38,215.1000,220.27,58046178.0,-1.28,0.0,1.28,1.171429,1.510000,0.775781,43.686734


In [12]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=1H4BLOX4T4ZVGAED'
r = requests.get(url)
data = r.json()

print(dict(data))

{'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'Nvidia Aggressively Bought, Russia Fires First Intercontinental Ballistic Missile, Adani Indicted - Apple  ( NASDAQ:AAPL ) ', 'url': 'https://www.benzinga.com/markets/cryptocurrency/24/11/42117158/nvidia-aggressively-bought-russia-fires-first-intercontinental-ballistic-missile-adani-ind', 'time_published': '20241121T162416', 'authors': ['The Arora Report'], 'summary': 'To gain an edge, this is what you need to know today. Please click here for an enlarged chart of NVIDIA Corp NVDA. Nvidia CEO Jensen Huang said, "The age of AI is in full steam, propelling a global shift to Nvidia computing." Blackwell demand is likely to exceed supply in FY26.', 'banner_image': 'https://thearorareport.com